In [ ]:
import pandas as pd
import numpy as np

from snowflake.snowpark import Session
import snowflake.snowpark as snowpark
from snowflake.snowpark.context import get_active_session
from snowflake.cortex import complete

from typing import List
import os
import sys
import json
import time
import requests

#Set up snowflake session vars and env vars
session = get_active_session()
session

In [ ]:
WITH RESULTS AS (SELECT 
    TIMESTAMP AS TS,
    RECORD_ATTRIBUTES:"snow.ai.observability.object.name" AS AGENT_NAME,
    RECORD_ATTRIBUTES:"snow.ai.observability.agent.thread_id" AS THREAD_ID,
    RECORD_ATTRIBUTES:"snow.ai.observability.agent.parent_message_id" AS PARENT_MESSAGE_ID,
    COALESCE(
        PARENT_MESSAGE_ID,
        LAST_VALUE (PARENT_MESSAGE_ID) IGNORE NULLS OVER (
          PARTITION BY AGENT_NAME,
          THREAD_ID
          ORDER BY
            TIMESTAMP ROWS BETWEEN UNBOUNDED PRECEDING
            AND CURRENT ROW
        )
      ) AS PARENT_MESSAGE,

    CONCAT(THREAD_ID, '-', PARENT_MESSAGE) AS THREAD_MESSAGE_ID,

    VALUE:"snow.ai.observability.request_body"."messages"[0]."content"[0]."text" AS INPUT_QUERY,
    RECORD_ATTRIBUTES:"snow.ai.observability.agent.planning.thinking_response" AS AGENT_PLANNING,
    RECORD_ATTRIBUTES:"snow.ai.observability.agent.tool.cortex_analyst.sql_query" AS GENERATED_SQL,
    RECORD_ATTRIBUTES:"snow.ai.observability.agent.tool.sql_execution.result" AS SQL_RESULT,
    RECORD_ATTRIBUTES:"snow.ai.observability.agent.tool.cortex_search.results" AS CORTEX_SEARCH_RESULT,
    RECORD_ATTRIBUTES:"snow.ai.observability.agent.tool.custom_tool.results" AS CUSTOM_TOOL_RESULT,
    RECORD_ATTRIBUTES:"ai.observability.record_root.output" AS AGENT_RESPONSE, 
    RECORD_ATTRIBUTES:"snow.ai.observability.agent.planning.model" AS REASONING_MODEL, 
    RECORD_ATTRIBUTES:"snow.ai.observability.agent.planning.tool.name" AS AVAILABLE_TOOLS, 
    RECORD_ATTRIBUTES:"snow.ai.observability.agent.planning.tool_selection.name" AS TOOL_SELECTION,
    CASE
        WHEN VALUE:"positive"='true' THEN 1
        WHEN VALUE:"positive"='false'THEN 0
        ELSE NULL
        END AS USER_FEEDBACK,
    VALUE:"feedback_message" AS USER_FEEDBACK_MESSAGE,

    RECORD:"name" as OPERATION,
    *
    FROM SNOWFLAKE.LOCAL.AI_OBSERVABILITY_EVENTS 
    WHERE SCOPE:name = 'snow.cortex.agent'
    AND AGENT_NAME = 'FINANCE_AGENT'
    AND OPERATION !='Agent'
    -- AND THREAD_ID = '475131216265'
    ORDER BY THREAD_ID, TIMESTAMP, START_TIMESTAMP ASC)

    SELECT * FROM RESULTS;

In [ ]:
WITH RESULTS AS (SELECT 
    TIMESTAMP AS TS,
    RECORD_ATTRIBUTES:"snow.ai.observability.object.name" AS AGENT_NAME,
    RECORD_ATTRIBUTES:"snow.ai.observability.agent.thread_id" AS THREAD_ID,
    RECORD_ATTRIBUTES:"snow.ai.observability.agent.parent_message_id" AS PARENT_MESSAGE_ID,
    COALESCE(
        PARENT_MESSAGE_ID,
        LAST_VALUE (PARENT_MESSAGE_ID) IGNORE NULLS OVER (
          PARTITION BY AGENT_NAME,
          THREAD_ID
          ORDER BY
            TIMESTAMP ROWS BETWEEN UNBOUNDED PRECEDING
            AND CURRENT ROW
        )
      ) AS PARENT_MESSAGE,

    CONCAT(THREAD_ID, '-', PARENT_MESSAGE) AS THREAD_MESSAGE_ID,

    VALUE:"snow.ai.observability.request_body"."messages"[0]."content"[0]."text" AS INPUT_QUERY,
    RECORD_ATTRIBUTES:"snow.ai.observability.agent.planning.thinking_response" AS AGENT_PLANNING,
    RECORD_ATTRIBUTES:"snow.ai.observability.agent.tool.cortex_analyst.sql_query" AS GENERATED_SQL,
    RECORD_ATTRIBUTES:"snow.ai.observability.agent.tool.sql_execution.result" AS SQL_RESULT,
    RECORD_ATTRIBUTES:"snow.ai.observability.agent.tool.cortex_search.results" AS CORTEX_SEARCH_RESULT,
    RECORD_ATTRIBUTES:"snow.ai.observability.agent.tool.custom_tool.results" AS CUSTOM_TOOL_RESULT,
    RECORD_ATTRIBUTES:"ai.observability.record_root.output" AS AGENT_RESPONSE, 
    RECORD_ATTRIBUTES:"snow.ai.observability.agent.planning.model" AS REASONING_MODEL, 
    RECORD_ATTRIBUTES:"snow.ai.observability.agent.planning.tool.name" AS AVAILABLE_TOOLS, 
    RECORD_ATTRIBUTES:"snow.ai.observability.agent.planning.tool_selection.name" AS TOOL_SELECTION,
    CASE
        WHEN VALUE:"positive"='true' THEN 1
        WHEN VALUE:"positive"='false'THEN 0
        ELSE NULL
        END AS USER_FEEDBACK,
    VALUE:"feedback_message" AS USER_FEEDBACK_MESSAGE,

    RECORD:"name" as OPERATION,
    *
    FROM SNOWFLAKE.LOCAL.AI_OBSERVABILITY_EVENTS 
    WHERE SCOPE:name = 'snow.cortex.agent'
    -- AND AGENT_NAME = 'FINANCE_AGENT'
    AND OPERATION !='Agent'
    -- AND THREAD_ID = '475131216265'
    ORDER BY THREAD_ID, TIMESTAMP, START_TIMESTAMP ASC)

    SELECT 
        THREAD_MESSAGE_ID,
        MIN(TIMESTAMP) AS TS,
        MIN(AGENT_NAME) AS AGENT_NAME,
        -- ARRAY_AGG(INPUT_QUERY ORDER BY TS ASC) AS INPUT_QUERY,
      ARRAY_AGG(INPUT_QUERY) WITHIN GROUP (ORDER BY TIMESTAMP ASC) AS INPUT_QUERIES,
        ARRAY_AGG(AGENT_RESPONSE)  WITHIN GROUP (ORDER BY TIMESTAMP ASC) AS AGENT_RESPONSE,
        ARRAY_AGG(AGENT_PLANNING)  WITHIN GROUP (ORDER BY TIMESTAMP ASC) AS AGENT_PLANNING,
        ARRAY_AGG(TOOL_SELECTION)  WITHIN GROUP (ORDER BY TIMESTAMP ASC) AS TOOL_CALLS,
        ARRAY_AGG(COALESCE(GENERATED_SQL, SQL_RESULT, CORTEX_SEARCH_RESULT, CUSTOM_TOOL_RESULT))  
        WITHIN GROUP (ORDER BY TIMESTAMP ASC) AS TOOL_RESULTS,
    
        FROM RESULTS
        
        GROUP BY THREAD_MESSAGE_ID
        ORDER BY TS ASC;

In [ ]:
import json

df = cell6.to_pandas()
df

# ARCHIVE BELOW

In [ ]:
SELECT SCOPE:name, COUNT(*) FROM SNOWFLAKE.LOCAL.AI_OBSERVABILITY_EVENTS 
-- WHERE
-- ORDER BY TIMESTAMP DESC
GROUP BY SCOPE
-- limit 10;